In [1]:
import pandas as pd
import osmium
import networkx as nx

# Datos - Latitud, Longitud - Municipios

In [2]:
df_municipios_ubicacion = pd.read_csv('../../Data/DIVIPOLA_Municipios.csv')

In [3]:
df_municipios_ubicacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1121 entries, 0 to 1120
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Código_Dep    1121 non-null   int64  
 1   Departamento  1121 non-null   object 
 2   Código_Mun    1121 non-null   int64  
 3   Municipio     1121 non-null   object 
 4   Tipo          1121 non-null   object 
 5   Latitud       1121 non-null   float64
 6   Longitud      1121 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 61.4+ KB


In [4]:
df_municipios_ubicacion.head()

,Código_Dep,Departamento,Código_Mun,Municipio,Tipo,Latitud,Longitud
0,5,ANTIOQUIA,5001,MEDELLÍN,Municipio,6.257590,-75.611031
1,5,ANTIOQUIA,5002,ABEJORRAL,Municipio,5.803728,-75.438474
2,5,ANTIOQUIA,5004,ABRIAQUÍ,Municipio,6.627569,-76.085978
3,5,ANTIOQUIA,5021,ALEJANDRÍA,Municipio,6.365534,-75.090597
4,5,ANTIOQUIA,5030,AMAGÁ,Municipio,6.032922,-75.708003


# Grafo

In [2]:
class CounterHandler(osmium.SimpleHandler):
    def __init__(self) -> None:
        osmium.SimpleHandler.__init__(self)
        self.num_nodes = 0
    
    def node(self, node):
        self.num_nodes += 1

In [3]:
handler = CounterHandler()

In [4]:
handler.apply_file('../../Data/colombia-latest.osm.pbf')

In [5]:
print(handler.num_nodes)

38269251


In [5]:
class RoadHandler(osmium.SimpleHandler):
    def __init__(self):
        super(RoadHandler, self).__init__()
        self.graph = nx.Graph()

    def way(self, w):
        if 'highway' in w.tags:
            road_id = w.id
            nodes = [node.ref for node in w.nodes]
            self.graph.add_edges_from(zip(nodes, nodes[1:]), road_id=road_id)


In [6]:
road_handler = RoadHandler()
road_handler.apply_file("../../Data/colombia-latest.osm.pbf")

In [7]:
mun_a = df_municipios_ubicacion.iloc[0]
point_a = (mun_a.iloc[5], mun_a.iloc[6])
mun_b = df_municipios_ubicacion.iloc[1]
point_b = (mun_b.iloc[5], mun_b.iloc[6])

point_a, point_b

((6.257590259, -75.61103107), (5.803728154, -75.43847353))

In [19]:
type(road_handler.graph.nodes)

networkx.classes.reportviews.NodeView

In [12]:
a = 0
for i in road_handler.graph.nodes:
    print(i)
    print(type(i))
    a += 1
    if a > 10:
        break

1110651131
<class 'int'>
1110651144
<class 'int'>
1778508096
<class 'int'>
9429606593
<class 'int'>
253762179
<class 'int'>
7985809281
<class 'int'>
5107946411
<class 'int'>
339058332
<class 'int'>
7985809282
<class 'int'>
5108012311
<class 'int'>
7990304983
<class 'int'>


In [ ]:
road_handler.graph.nodes

In [8]:
nearest_node_to_a = min(road_handler.graph.nodes, key=lambda node: (node[0] - point_a[0])**2 + (node[1] - point_a[1])**2)
# nearest_node_to_b = min(road_handler.graph.nodes, key=lambda node: (node[0] - point_b[0])**2 + (node[1] - point_b[1])**2)

TypeError: 'int' object is not subscriptable

In [9]:
shortest_path = nx.shortest_path(road_handler.graph, source=nearest_node_to_a, target=nearest_node_to_b)

NameError: name 'nearest_node_to_a' is not defined

In [ ]:
road_distance = sum(road_handler.graph.get_edge_data(u, v)["length"] for u, v in zip(shortest_path, shortest_path[1:]))
